<a href="https://colab.research.google.com/github/smarginatura/makemore/blob/dev/rnn_bs_lr_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/smarginatura/makemore.git

Cloning into 'makemore'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 116 (delta 38), reused 57 (delta 30), pack-reused 42
Receiving objects: 100% (116/116), 315.62 KiB | 3.90 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [2]:
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.2/289.2 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [3]:
import wandb
from google.colab import userdata
wandb.login(key=userdata.get('WANDB_KEY2'))

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
%cd makemore

/content/makemore


In [6]:
import os, sys, time, math, random

import numpy as np
import torch
from torch.nn import functional as F

from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

from tqdm.auto import tqdm

from model import ModelConfig, RNN, loss_fn
from data_loader import create_datasets, InfiniteDataLoader

# Utils
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# Sweep

In [7]:
def train(config=None):

    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        set_seed(config.seed)
        os.makedirs(config.work_dir, exist_ok=True)

        @torch.inference_mode()
        def evaluate(model, dataset, batch_size=50, max_batches=None):
            model.eval()
            loader = DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=0)
            losses = []
            for i, batch in enumerate(loader):
                batch = [t.to(config.device) for t in batch]
                X, Y = batch
                logits = model(X)
                loss = loss_fn(logits, Y)
                losses.append(loss.item())
                if max_batches is not None and i >= max_batches:
                    break
            mean_loss = torch.tensor(losses).mean().item()
            model.train() # reset model back to training mode
            return mean_loss

        # init datasets
        train_dataset, test_dataset = create_datasets(config.input_file)
        vocab_size = train_dataset.get_vocab_size()
        block_size = train_dataset.get_output_length()
        print(f"\ndataset determined that: {vocab_size=}, {block_size=}")

        # init model
        model_config = ModelConfig(
            vocab_size=vocab_size,
            block_size=block_size,
            n_layer=config.n_layer,
            n_embd=config.n_embd,
            n_embd2=config.n_embd2)
        model = RNN(model_config, cell_type=config.cell_type)
        model.to(config.device)
        print(f"model #params: {sum(p.numel() for p in model.parameters())}")

        # init optimizer
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=config.learning_rate,
            weight_decay=config.weight_decay,
            betas=(0.9, 0.99),eps=1e-8,
            )

        # init dataloader
        batch_loader = InfiniteDataLoader(
            train_dataset,
            batch_size=config.batch_size,
            pin_memory=True,
            num_workers=config.num_workers)

        best_loss = None

        for step in tqdm(range(config.max_steps)):

            t0 = time.time()

            batch = batch_loader.next()
            batch = [t.to(config.device) for t in batch]
            X, Y = batch
            logits = model(X)
            loss = loss_fn(logits, Y)
            model.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()
            # wait for all CUDA work on the GPU to finish then calculate iteration time taken
            if config.device.startswith('cuda'):
                torch.cuda.synchronize()

            t1 = time.time()

            # Evaluate the model
            if step > 0 and step % 500 == 0:
                train_loss = evaluate(model, train_dataset, batch_size=100, max_batches=10)
                test_loss  = evaluate(model, test_dataset,  batch_size=100, max_batches=10)
                wandb.log({
                    "train/loss": train_loss,
                    "test/loss": test_loss,
                }, step=step)
                print(f"step {step} train loss: {train_loss:.4f} test loss: {test_loss:.4f}")

                # Save the model to disk if it has improved
                if best_loss is None or test_loss < best_loss:
                    out_path = os.path.join(config.work_dir, "model.pt")
                    print(f"test loss {test_loss:4f} is the best so far, saving model to {out_path}")
                    torch.save(model.state_dict(), out_path)
                    best_loss = test_loss

In [8]:
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'test/loss',
        'goal': 'minimize'
        }
    }

parameters_dict = {
        'learning_rate': {
            'values': [1e-3]
            },
        'batch_size': {
            'values': [32, 64, 128, 256, 512]
            },}
parameters_dict.update({
    'n_layer': {'value': 4},
    'n_embd': {'value' : 64},
    'n_embd2': {'value' :64},
    'cell_type': {'value': 'rnn'},
    'weight_decay': {'value': 0.01},
    'input_file': {'value': 'data/names.txt'},
    'work_dir': {'value': 'out'},
    'num_workers': {'value': 2},
    'max_steps': {'value': 50_000},
    'device':{'value':'cuda' if torch.cuda.is_available() else 'cpu'},
    'seed': {'value': 3407},
    'top_k': {'value':  -1},
    })
sweep_config['parameters'] = parameters_dict

In [9]:
import pprint
pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'test/loss'},
 'parameters': {'batch_size': {'values': [32, 64, 128, 256, 512]},
                'cell_type': {'value': 'rnn'},
                'device': {'value': 'cuda'},
                'input_file': {'value': 'data/names.txt'},
                'learning_rate': {'values': [0.001]},
                'max_steps': {'value': 50000},
                'n_embd': {'value': 64},
                'n_embd2': {'value': 64},
                'n_layer': {'value': 4},
                'num_workers': {'value': 2},
                'seed': {'value': 3407},
                'top_k': {'value': -1},
                'weight_decay': {'value': 0.01},
                'work_dir': {'value': 'out'}}}


In [10]:
sweep_id = wandb.sweep(sweep_config, project="char-rnn-bs-sweeps-50000-steps")

Create sweep with ID: 3dw7opq1
Sweep URL: https://wandb.ai/polyphony/char-rnn-bs-sweeps-50000-steps/sweeps/3dw7opq1


In [11]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: ghzmu5zs with config:
wandb: 	batch_size: 32
wandb: 	cell_type: rnn
wandb: 	device: cuda
wandb: 	input_file: data/names.txt
wandb: 	learning_rate: 0.001
wandb: 	max_steps: 50000
wandb: 	n_embd: 64
wandb: 	n_embd2: 64
wandb: 	n_layer: 4
wandb: 	num_workers: 2
wandb: 	seed: 3407
wandb: 	top_k: -1
wandb: 	weight_decay: 0.01
wandb: 	work_dir: out
wandb: Currently logged in as: smarginatura (polyphony). Use `wandb login --relogin` to force relogin


number of examples in the dataset: 32033
max word length: 15
number of unique characters in the vocabulary: 26
vocabulary:
abcdefghijklmnopqrstuvwxyz
split up the dataset into 31033 training examples and 1000 test examples

dataset determined that: vocab_size=27, block_size=16
model #params: 11803


  0%|          | 0/50000 [00:00<?, ?it/s]

step 500 train loss: 2.2501 test loss: 2.2297
test loss 2.229686 is the best so far, saving model to out/model.pt
step 1000 train loss: 2.2134 test loss: 2.1800
test loss 2.180049 is the best so far, saving model to out/model.pt
step 1500 train loss: 2.1632 test loss: 2.1539
test loss 2.153885 is the best so far, saving model to out/model.pt
step 2000 train loss: 2.1263 test loss: 2.1346
test loss 2.134612 is the best so far, saving model to out/model.pt
step 2500 train loss: 2.1561 test loss: 2.1211
test loss 2.121068 is the best so far, saving model to out/model.pt
step 3000 train loss: 2.1335 test loss: 2.1119
test loss 2.111885 is the best so far, saving model to out/model.pt
step 3500 train loss: 2.1088 test loss: 2.1016
test loss 2.101563 is the best so far, saving model to out/model.pt
step 4000 train loss: 2.1096 test loss: 2.1020
step 4500 train loss: 2.1060 test loss: 2.0904
test loss 2.090446 is the best so far, saving model to out/model.pt
step 5000 train loss: 2.0930 test 

test/loss,█▅▄▄▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▂▁▂▁▁▂▁▁▁▂▁▁▂▁▂▂▂
test/loss,2.03571
train/loss,2.02062


...

number of examples in the dataset: 32033
max word length: 15
number of unique characters in the vocabulary: 26
vocabulary:
abcdefghijklmnopqrstuvwxyz
split up the dataset into 31033 training examples and 1000 test examples

dataset determined that: vocab_size=27, block_size=16
model #params: 11803


  0%|          | 0/50000 [00:00<?, ?it/s]

step 500 train loss: 2.1612 test loss: 2.1409
test loss 2.140901 is the best so far, saving model to out/model.pt
step 1000 train loss: 2.1200 test loss: 2.0896
test loss 2.089609 is the best so far, saving model to out/model.pt
step 1500 train loss: 2.0793 test loss: 2.0673
test loss 2.067284 is the best so far, saving model to out/model.pt
step 2000 train loss: 2.0443 test loss: 2.0553
test loss 2.055260 is the best so far, saving model to out/model.pt
step 2500 train loss: 2.0597 test loss: 2.0459
test loss 2.045858 is the best so far, saving model to out/model.pt
step 3000 train loss: 2.0437 test loss: 2.0398
test loss 2.039826 is the best so far, saving model to out/model.pt
step 3500 train loss: 2.0307 test loss: 2.0375
test loss 2.037510 is the best so far, saving model to out/model.pt
step 4000 train loss: 2.0289 test loss: 2.0328
test loss 2.032790 is the best so far, saving model to out/model.pt
step 4500 train loss: 2.0256 test loss: 2.0314
test loss 2.031450 is the best so 

test/loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▅▄▄▃▂▃▂▂▂▂▂▁▂▂▂▂▂▁▂▂▂▂▂▁▂▂▁▂▁▁▁▂▂▂▂▁▂▂▂
test/loss,2.0169
train/loss,1.97757


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
